In [8]:
!pip install transformers


In [18]:
from transformers import TFAutoModel,AutoTokenizer
!pip install datasets
from datasets import load_dataset
import tensorflow as tf


In [33]:
 model=TFAutoModel.from_pretrained("bert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [34]:
tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")

In [12]:
inputs=tokenizer(['hello world','hi i hate you so much'],padding=True,truncation=True,return_tensors='tf')

In [35]:
output=model(inputs)

In [36]:
emotions=load_dataset('SetFit/emotion')

Repo card metadata block was not found. Setting CardData to empty.


In [37]:
emotions

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2000
    })
})

In [38]:
def tokenize(batch):
  return tokenizer(batch["text"],padding=True,truncation=True)

In [39]:
emotions_encoded=emotions.map(tokenize,batched=True,batch_size=None)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

In [40]:
emotions_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [45]:
#Converting from HF format to TF format
emotions_encoded.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])
BATCH_SIZE=64

def order(inp):
  '''
  groups all ip of bert into a single dict and then op it w labels
  '''
  data=list(inp.values())
  return{
      'input_ids':data[1],
      'attention_mask':data[2],
      'token_type_ids':data[3]
  },data[0]

#converting train split of emotions encoded to tf format
train_dataset=tf.data.Dataset.from_tensor_slices(emotions_encoded['train'][:])

#Set batchsize and shuffle
train_dataset=train_dataset.batch(BATCH_SIZE).shuffle(1000)

#map the order funciton
train_dataset=train_dataset.map(order,num_parallel_calls=tf.data.AUTOTUNE)



test_dataset=tf.data.Dataset.from_tensor_slices(emotions_encoded['test'][:])
test_dataset=test_dataset.batch(BATCH_SIZE)
test_dataset=test_dataset.map(order,num_parallel_calls=tf.data.AUTOTUNE)





In [46]:
class BERTForClassification(tf.keras.Model):

  def __init__(self,bert_model,num_classes):
    super().__init__()
    self.bert=bert_model
    self.fc=tf.keras.layers.Dense(num_classes,activation='softmax')

  def call(self,inputs):
    x=self.bert(inputs)[1]#returns hidden state and pooler output but we need only pooler so [1]
    return self.fc(x)


In [47]:
classifier=BERTForClassification(model,num_classes=6)
classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
    )

In [48]:
Res=classifier.fit(
    train_dataset,
    epochs=6
)

Epoch 1/6
250/250 [==============================] - 333s 1s/step - loss: 1.0252 - accuracy: 0.6091
Epoch 2/6
250/250 [==============================] - 282s 1s/step - loss: 0.2548 - accuracy: 0.9076
Epoch 3/6
250/250 [==============================] - 282s 1s/step - loss: 0.1566 - accuracy: 0.9365
Epoch 4/6
250/250 [==============================] - 281s 1s/step - loss: 0.1164 - accuracy: 0.9463
Epoch 5/6
250/250 [==============================] - 282s 1s/step - loss: 0.0994 - accuracy: 0.9521
Epoch 6/6
250/250 [==============================] - 282s 1s/step - loss: 0.0913 - accuracy: 0.9563


In [49]:
classifier.evaluate(test_dataset)


32/32 [==============================] - 12s 276ms/step - loss: 0.1724 - accuracy: 0.9265


[0.17238929867744446, 0.9265000224113464]